# Applied Text and Natural Language Analytics, Fall 2020

### Assignment 5

Submitted by - 
Harsh Dhanuka, hd2457

In [1]:
import json

from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

import spacy
from spacy.gold import docs_to_json
from spacy.util import minibatch, compounding
from spacy.pipeline import SentenceSegmenter
import random

import urllib.request as url

from bs4 import BeautifulSoup
from bs4.element import Comment

import warnings
warnings.filterwarnings("ignore")

from collections import OrderedDict
import numpy as np
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

from sumy.parsers.plaintext import PlaintextParser
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [2]:
nlp = spacy.load('en_core_web_sm')

I am selecting the 'Netflix' dataset from  Assignment 2 for the analysis.

The random news article from this dataset is: http://omgili.com/ri/.wHSUbtEfZSCvFgWhG.N__Y_kk6rEaYdjsrpI1bEeKktgDgldQ2s_6MRmlv4jK7fMNVEhQqU4EXb61rVFualijBYg7orEsJZJyBrzqNk1FQ_2z6DFSGmxcGnRpX.EB__lgRE2mriFDCyaZoJ1tyJAvD8XG37Dct6_XhBQCreko_WuncGLV.lltj.I7NCcFr0  

This site now automatically redirects to the link - https://www.stuff.co.nz/entertainment/entertainment-top-stories/300026292/judge-gives-control-of-tiger-king-joe-exotics-zoo-to-carole-baskin 

I will use this new link for my analysis.

In [3]:
# Define 

link = 'https://www.stuff.co.nz/entertainment/entertainment-top-stories/300026292/judge-gives-control-of-tiger-king-joe-exotics-zoo-to-carole-baskin'

In [4]:
html = url.urlopen(link)
raw = html.read()

In [5]:
# Using the code provided by lecturer in Module 3 class excercise to parse text from a web or .html article

def tag_visible(element):
    if element.parent.name in ['style', 'script', 
                               # 'head', 'title', 
                               'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

text = text_from_html(raw)
print(text[:1000])

                           Judge gives control of 'Tiger King' Joe Exotic's zoo to Carole Baskin | Stuff.co.nz           news National World Coronavirus Climate Change Politics Business prosper Farming Technology Sport Rugby voices & in depth perspectives Pou Tiaki Spotlight Stuff Nation Cartoons KEA Kids News living Travel Homed Life & Style Entertainment bravo Motoring Food & Wine Oddstuff regions northland Auckland Waikato Bay of Plenty Taranaki hawke's bay manawatu wellington nelson marlborough canterbury south cantebury otago southland more Weather Quizzes Puzzles Newsletters about stuff contribute Advertising Careers Privacy Contact stuff family Play Stuff neighbourly mags4gifts stuff events stuff coupons Entertainment Entertainment Top Stories Judge gives control of 'Tiger King' Joe Exotic's zoo to Carole Baskin 08:27, Jun 03 2020 Facebook Twitter Whats App Reddit Email A federal judge in Oklahoma has awarded ownership of the zoo made famous in Netflix's Tiger King docuseries to

#### The above is the raw text which I will be using for passing to the SVO funstions.

# 1. Extract and print subject-verb-object (SVO) relations from each sentence

In [6]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]

In [7]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [8]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [9]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [10]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [11]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [12]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [13]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [14]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [15]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [16]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [17]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [18]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [19]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs():
    #nlp = English()

    tok = nlp(text)
    svos = findSVOs(tok)
    #printDeps(tok)
    print(svos)

In [20]:
print(" ")
print("The Subject-Verb-Object relations for all the sentences in the article are as follows: ")
print(" ")

if __name__ == "__main__":
    testSVOs()

 
The Subject-Verb-Object relations for all the sentences in the article are as follows: 
 
[('judge', 'gives', 'control'), ('family', 'play', 'stuff'), ('family', 'play', 'coupons'), ('judge', 'gives', 'control'), ('judge', 'awarded', 'ownership'), ('ownership', 'made', 'famous'), ('palk', 'granted', 'control'), ('passage', 'serving', 'term'), ('term', 'killing', 'tigers'), ('who', 'play', 'him'), ('cage', 'playing', 'exotic'), ('baskin', 'sued', 'passage'), ('attempt', 'paying', 'judgment'), ('attempt', 'paying', 'judgment'), ('baskin', 'take', 'control'), ('attorneys', 'representing', 'corporation'), ('attorneys', '!return', 'messages'), ('messages', 'seeking', 'comment'), ('he', 'repeated', 'plea'), ('woman', 'employs', 'man'), ('man', 'using', 'box'), ('machine', 'outperforms', 'maker'), ('teacher', 'speeding', 'van'), ('driver', 'passes', 'man'), ('couple', 'found', 'blacks'), ('campese', 'warns', 'nz')]


# 2. Apply TextRank for ranking and selecting key phrases, print the result

In [21]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [22]:
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN',"ADP"], window_size=8, lower=False)

print(" ")
print("The Key Phrases according to their ranks for all the sentences in the article are as follows: ")
print(" ")

tr4w.get_keywords(20)

 
The Key Phrases according to their ranks for all the sentences in the article are as follows: 
 
zoo - 3.0978807385959084
Exotic - 2.906616077807699
Joe - 2.872306668835595
Maldonado - 2.585021255096602
Passage - 2.534808113449315
Baskin - 2.312616952028586
stuff - 1.8689037058692453
Australia - 1.8431370298801388
Monday - 1.7884209487652747
Stuff - 1.767459044284851
Entertainment - 1.7580346094613186
Tiger - 1.7404878234936123
King - 1.732044704858808
control - 1.6429326469157965
judgment - 1.4917722234350905
Privacy - 1.4369425869162553
Advertising - 1.4187858520012622
Carole - 1.4034608772485013
Oklahoma - 1.3632783769123566
Careers - 1.35557411287853
Judge - 1.3260157032129625
Canterbury - 1.2347537890409686


### Alternate Method

In [23]:
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):
    import itertools, nltk, string
    
    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent)) for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  for key, group in itertools.groupby(all_chunks, lambda word__pos__chunk: word__pos__chunk[2] != 'O') if key]

    return [cand for cand in candidates
            if cand not in stop_words and not all(char in punct for char in cand)]

In [24]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, nltk, string

    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize and POS-tag words
    tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent)
                                                                    for sent in nltk.sent_tokenize(text)))
    # filter on certain POS tags and lowercase all words
    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

In [25]:
def score_keyphrases_by_tfidf(texts, candidates='chunks'):
    import gensim, nltk
    
    # extract candidates from each text in texts, either chunks or words
    if candidates == 'chunks':
        boc_texts = [extract_candidate_chunks(text) for text in texts]
    elif candidates == 'words':
        boc_texts = [extract_candidate_words(text) for text in texts]
    # make gensim dictionary and corpus
    dictionary = gensim.corpora.Dictionary(boc_texts)
    corpus = [dictionary.doc2bow(boc_text) for boc_text in boc_texts]
    # transform corpus with tf*idf model
    tfidf = gensim.models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    return corpus_tfidf, dictionary

In [26]:
def score_keyphrases_by_textrank(text, n_keywords=0.05):
    from itertools import takewhile, tee
    import operator
    import networkx, nltk
    
    # tokenize for all words, and extract *candidate* words
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]
    candidates = extract_candidate_words(text)
    # build graph, each node is a unique candidate
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates))
    # iterate over word-pairs, add unweighted edges into graph
    def pairwise(iterable):
        """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    for w1, w2 in pairwise(candidates):
        if w2:
            graph.add_edge(*sorted([w1, w2]))
    # score nodes using default pagerank algorithm, sort by score, keep top n_keywords
    ranks = networkx.pagerank(graph)
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)[:n_keywords]}
                  #for word_rank in sorted(ranks.iteritems(), key=lambda x: x[1], reverse=True)[:n_keywords]}
                  
    #sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[:3]
    keywords = set(word_ranks.keys())
    # merge keywords into keyphrases
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            # counter as hackish way to ensure merged keyphrases are non-overlapping
            j = i + len(kp_words)
            
    return sorted(keyphrases.items(), key=operator.itemgetter(1), reverse=True)
    #return sorted(keyphrases.iteritems(), key=lambda x: x[1], reverse=True)

In [27]:
print(" ")
print("The Key Phrases according to their ranks for all the sentences in the article are as follows: ")
print(" ")

df = pd.DataFrame(score_keyphrases_by_textrank(text))
df.columns = ['Key Word / Phrase', 'Rank Score']
df

 
The Key Phrases according to their ranks for all the sentences in the article are as follows: 
 


,Key Word / Phrase,Rank Score
0,stuff,0.017673
1,maldonado-passage,0.014362
2,baskin,0.012792
3,zoo,0.012561
4,news,0.010272
5,joe exotic,0.009716
6,monday,0.009134
7,judge,0.008648
8,entertainment,0.008489
9,entertainment entertainment,0.008489


It is to note that index 9 has a keyword which is twice of itself.

# 3. Apply LexRank to produce an extractive summary of 5 sentences

In [28]:
class TextSummary(object):

    def __init__(self, feeds_str, num_sents):
        self.summary = str()
        
        parser = PlaintextParser.from_string(feeds_str, Tokenizer("english"))
        summarizer = LexRankSummarizer()

        sentences = summarizer(parser.document, num_sents)  # Summarize the document with 5 sentences
        for sentence in sentences:
            self.summary += (sentence.__unicode__())

    def output(self):
        return self.summary

In [29]:
text_to_sum = TextSummary(text,5)
print(text_to_sum.output())

Judge gives control of 'Tiger King' Joe Exotic's zoo to Carole Baskin | Stuff.co.nz           news National World Coronavirus Climate Change Politics Business prosper Farming Technology Sport Rugby voices & in depth perspectives Pou Tiaki Spotlight Stuff Nation Cartoons KEA Kids News living Travel Homed Life & Style Entertainment bravo Motoring Food & Wine Oddstuff regions northland Auckland Waikato Bay of Plenty Taranaki hawke's bay manawatu wellington nelson marlborough canterbury south cantebury otago southland more Weather Quizzes Puzzles Newsletters about stuff contribute Advertising Careers Privacy Contact stuff family Play Stuff neighbourly mags4gifts stuff events stuff coupons Entertainment Entertainment Top Stories Judge gives control of 'Tiger King' Joe Exotic's zoo to Carole Baskin 08:27, Jun 03 2020 Facebook Twitter Whats App Reddit Email A federal judge in Oklahoma has awarded ownership of the zoo made famous in Netflix's Tiger King docuseries to Joe Exotic's chief rival.I